In [1]:
import pandas as pd
import geopandas as gpd
import folium
import shapely
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
d10 = pd.read_csv("../data/original/10.오산시_학교위치정보.csv")
d11 = gpd.read_file("../data/original/11.오산시_초등학교_통학구.geojson")
d12 = gpd.read_file("../data/original/12.오산시_중학교_학군.geojson")
d13 = pd.read_csv("../data/original/13.오산시_어린이집_유치원현황.csv")
d14 = pd.read_csv("../data/original/14.오산시_기상데이터(2010~2019).csv")
d15 = pd.read_csv("../data/original/15.오산시_무인교통단속카메라.csv")
d16 = pd.read_csv("../data/original/16.오산시_도로안전표지표준데이터.csv")
d17 = gpd.read_file("../data/original/17.오산시_횡단보도.geojson")
d18 = pd.read_csv("../data/original/18.오산시_과속방지턱표준데이터.csv")

## d10 - 학교 위치 정보

In [3]:
d10.head()

,학교구분,학교명,학교위치_경도,학교위치_위도
0,초등학교,원동초등학교,127.069699,37.135168
1,초등학교,운산초등학교,127.077414,37.152500
2,초등학교,화성초등학교,127.068858,37.164615
3,중학교,대호중학교,127.056978,37.161864
4,중학교,매홀중학교,127.068500,37.167778


학교구분과 학교 이름, 학교의 위치(경도, 위도)를 갖고 있다

In [4]:
d10.isna().sum()

학교구분       0
학교명        0
학교위치_경도    0
학교위치_위도    0
dtype: int64

na값은 없다

In [5]:
d10.shape

(41, 4)

In [6]:
for col in d10.columns:
    print(len(d10[col].value_counts()))

3
41
40
40


겹치는 학교이름명은 없지만 겹치는 위도는 있다..?

In [7]:
d10["학교위치_경도"].value_counts()[d10["학교위치_경도"].value_counts()>1]

127.061305    2
Name: 학교위치_경도, dtype: int64

In [8]:
d10["학교위치_위도"].value_counts()[d10["학교위치_위도"].value_counts()>1]

37.153004    2
Name: 학교위치_위도, dtype: int64

In [9]:
d10[d10["학교위치_경도"] == 127.0613054]

,학교구분,학교명,학교위치_경도,학교위치_위도
29,중학교,오산중학교,127.061305,37.153004
32,고등학교,오산고등학교,127.061305,37.153004


붙어있는 중,고등학교였다!

In [10]:
d10["학교구분"].value_counts()

초등학교    24
중학교      9
고등학교     8
Name: 학교구분, dtype: int64

이정도 있다

In [11]:
m10 = folium.Map(location = [37.16501470733718, 127.05422687735454], tiles = 'openstreetmap', zoom_start = 12)

for i in range(0, len(d10)):
    folium.Circle(
        location = [d10.iloc[i]["학교위치_위도"], d10.iloc[i]["학교위치_경도"]],
        radius = 30,
        color = "red",
    ).add_to(m10)

In [12]:
m10

뭔가 철도따라 분포해있는 느낌이다

In [13]:
def generate_color(school):
    if school == "초등학교":
        return "red"
    elif school == "중학교":
        return "blue"
    else:
        return "black"

In [14]:
m10_2 = folium.Map(location = [37.16501470733718, 127.05422687735454], tiles = 'openstreetmap', zoom_start = 12)

for i in range(0, len(d10)):
    folium.Circle(
        location = [d10.iloc[i]["학교위치_위도"], d10.iloc[i]["학교위치_경도"]],
        radius = 30,
        color = generate_color(d10.iloc[i]["학교구분"]),
    ).add_to(m10_2)

In [15]:
m10_2

같은 종류의 학교보다는 다른 종류의 학교가 붙어있는 경우가 많다

## d11 - 오산시_초등학교_통학구

In [16]:
d11.head()

,OBJECTID,학구ID,학구명,학구분류,geometry
0,2648,Z000102542,오산고현초통학구역,0,"MULTIPOLYGON (((127.08560 37.13923, 127.08593 ..."
1,2774,Z000102531,수청초통학구역,0,"MULTIPOLYGON (((127.06076 37.17523, 127.06074 ..."
2,3259,Z000150216,대호초성산초공동통학구역,1,"MULTIPOLYGON (((127.03921 37.16869, 127.03953 ..."
3,3300,Z000102507,문시초통학구역,0,"MULTIPOLYGON (((127.05068 37.17408, 127.05084 ..."
4,3299,Z000102567,필봉초통학구역,0,"MULTIPOLYGON (((127.05151 37.18063, 127.05274 ..."


In [17]:
d11.shape

(31, 5)

- objectid : 데이터의 id값이므로 딱히 필요는 없을듯
- 학구id, 학구명 : 학구명과 고유 id 
- 학구분류 : 학구분류 정보 (---초 통학구역 / ---초 공동통학구역)
    - 공동통학구역 : 특정 지역 내 학생이 복수의 초등학교 가운데 하나를 선택하여 입학하거나 전학할 수 있도록 설정한 통학구역
    - 통학구역 : 특정 지역 주민의 자녀에게 특정한 학교에 갈 것을 지정해 놓은 구역. 원거리 통학을 억제하고 취학 아동 수가 지역별로 균형을 이루도록 초등학교, 중등학교 따위에서 구역을 제한하고 있다.

In [18]:
for col in d11.columns:
    if col!="geometry":
        print(len(d11[col].value_counts()))

31
31
31
2


- objectid, 학구id, 학구명은 모두 고유한 값으로 그냥 셋 다 같은 것으로 보면 됨

In [19]:
d11["학구분류"].value_counts()

0    23
1     8
Name: 학구분류, dtype: int64